<a href="https://colab.research.google.com/github/asalunai/IoT-trabalho/blob/main/teste_ana.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# -*- coding: utf-8 -*-

import numpy as np
import pandas as pd
import requests 

In [4]:
def baixa(est, datai, dataf):

  # baixa os dados e converte em JSON:
  url = 'https://apitempo.inmet.gov.br/estacao/' + datai + '/' + dataf + '/' + est
  r = requests.get(url)
  info = r.json()
  # print(info[0])

  # transforma os dados em JSON em Dataframe:
  df = pd.DataFrame(info)
  colunas = ['DT_MEDICAO', 'HR_MEDICAO','TEM_MAX','TEM_MIN']
  df = df[colunas]

  # formata data:
  df['HR_MEDICAO'] = df['HR_MEDICAO'].astype(str).str.zfill(4)
  df['HR_MEDICAO'] = df['HR_MEDICAO'].str[:2]
  df['DT_MEDICAO'] = df['DT_MEDICAO'] + ' ' + df['HR_MEDICAO'] + ':00'
  df.drop('HR_MEDICAO', axis=1, inplace=True)

  # Define a data da medição como index:
  df.set_index('DT_MEDICAO', inplace=True)
  df.index = pd.to_datetime(df.index)

  # calcula a temperatura média:
  df = df.apply(pd.to_numeric, errors='coerce')
  df['TEM_MED'] = (df['TEM_MAX'] + df['TEM_MIN']) / 2
  df.drop(['TEM_MAX', 'TEM_MIN'], axis=1, inplace=True)

  # calcula máximas e mínimas diárias:
  df_med = df.resample('D').count()
  df_med.columns = ['Contagem']
  df_med['TP_MAX'] = df['TEM_MED'].resample('D').max()
  df_med['TP_MIN'] = df['TEM_MED'].resample('D').min()

  # se um dia apresentar menos do que 75% dos dados esperados, 
  # então considere esse dia como sendo não-representativo 
  # para o cálculo das máximas e mínimas diárias:
  #df_med.iloc[2,0] = 3 #teste
  df_med.loc[(df_med.Contagem < 24*0.75), ['TP_MAX', 'TP_MIN']] = np.nan

  # Salva o arquivo em um CSV
  df_med.to_csv(est+'.csv', float_format='%.3f')

  return

In [6]:
# Formato da data: AAAA-MM-DD
datai = '2019-09-01'
dataf = '2020-09-30'
est_cod = ['A621', 'A618', 'A606', 'A609', 'A607']
nomes = ['Vila Militar', 'Teresópolis', 'Cabo Frio', 'Resende', 'Campos dos Goytacazes']

for est in est_cod:
  print(est)
  baixa(est, datai, dataf)


A621
A618
A606
A609
A607
